In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/sales_subset.csv")

In [3]:
df.head()

Unnamed: 0  store type  department        date  weekly_sales  is_holiday  \
0           0      1    A           1  2010-02-05      24924.50       False   
1           1      1    A           1  2010-03-05      21827.90       False   
2           2      1    A           1  2010-04-02      57258.43       False   
3           3      1    A           1  2010-05-07      17413.94       False   
4           4      1    A           1  2010-06-04      17558.09       False   

   temperature_c  fuel_price_usd_per_l  unemployment  
0       5.727778              0.679451         8.106  
1       8.055556              0.693452         8.106  
2      16.816667              0.718284         7.808  
3      22.527778              0.748928         7.808  
4      27.050000              0.714586         7.808

In [4]:
df.info()
df['weekly_sales'].mean()
df['weekly_sales'].median()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10774 entries, 0 to 10773
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            10774 non-null  int64  
 1   store                 10774 non-null  int64  
 2   type                  10774 non-null  object 
 3   department            10774 non-null  int64  
 4   date                  10774 non-null  object 
 5   weekly_sales          10774 non-null  float64
 6   is_holiday            10774 non-null  bool   
 7   temperature_c         10774 non-null  float64
 8   fuel_price_usd_per_l  10774 non-null  float64
 9   unemployment          10774 non-null  float64
dtypes: bool(1), float64(4), int64(3), object(2)
memory usage: 768.2+ KB


12049.064999999999

In [5]:
df['date'].max()
df['date'].min()

'2010-02-05'

In [6]:
#Using the function below, calculate the IQR when your data contains outliers
def iqr(column):
    return column.quantile(0.75) - column.quantile(0.25)

print(df[['temperature_c','fuel_price_usd_per_l', 'unemployment']].agg(iqr))

temperature_c           16.583333
fuel_price_usd_per_l     0.073176
unemployment             0.565000
dtype: float64


In [7]:
print(df[['temperature_c','fuel_price_usd_per_l','unemployment']].agg([iqr, np.median]))

        temperature_c  fuel_price_usd_per_l  unemployment
iqr         16.583333              0.073176         0.565
median      16.966667              0.743381         8.099


In [8]:
#Cumulative statistics
df = df.sort_values('date')
#print(df)
df['cum_weekly_sales'] = df['weekly_sales'].cumsum()
#print(df)
df['cum_max_sales'] = df['weekly_sales'].cummax()
#print(df)
print(df[['date','weekly_sales','cum_weekly_sales','cum_max_sales']])

             date  weekly_sales  cum_weekly_sales  cum_max_sales
0      2010-02-05      24924.50      2.492450e+04       24924.50
6437   2010-02-05      38597.52      6.352202e+04       38597.52
1249   2010-02-05       3840.21      6.736223e+04       38597.52
6449   2010-02-05      17590.59      8.495282e+04       38597.52
6461   2010-02-05       4929.87      8.988269e+04       38597.52
...           ...           ...               ...            ...
3592   2012-10-05        440.00      2.568932e+08      293966.05
8108   2012-10-05        660.00      2.568938e+08      293966.05
10773  2012-10-05        915.00      2.568947e+08      293966.05
6257   2012-10-12          3.00      2.568947e+08      293966.05
3384   2012-10-26        -21.63      2.568947e+08      293966.05

[10774 rows x 4 columns]


In [9]:
#Summarize Categorical data
#df.drop_duplicates(subset=['store','department'])
store_types = df.drop_duplicates(subset=['store','type'])
store_depts= df.drop_duplicates(subset=['store','department'])
holiday_dates = df[df['is_holiday']].drop_duplicates('date')
print(holiday_dates)

      Unnamed: 0  store type  department        date  weekly_sales  \
5198        5198     13    A          78  2010-02-12         12.00   
5606        5606     14    A          18  2010-09-10          1.94   
6810        6810     19    A          47  2010-12-31       -449.00   
6820        6820     19    A          48  2011-09-09        197.00   
4295        4295     10    B          77  2011-11-25       1590.00   
6815        6815     19    A          47  2012-02-10         15.00   
8072        8072     20    A          96  2012-09-07         -8.97   

      is_holiday  temperature_c  fuel_price_usd_per_l  unemployment  \
5198        True       0.644444              0.705604         8.316   
5606        True      21.594444              0.713001         8.743   
6810        True      -1.861111              0.881278         8.067   
6820        True      20.155556              1.038197         7.806   
4295        True      15.933333              0.993287         7.874   
6815        T

In [10]:
df['store'].value_counts(sort=True)
#normalize argument can turn the count into proportions of the total
df['store'].value_counts(normalize=True)
df.groupby('store')['weekly_sales'].mean()
df.groupby('department')['weekly_sales'].agg([min,max,sum])
df.groupby(['store','department'])['weekly_sales'].mean()
df.groupby(['store','department'])[['weekly_sales','unemployment']].mean()

weekly_sales  unemployment
store department                            
1     1           23491.755000      7.854417
      2           47421.124167      7.854417
      3           12872.590000      7.854417
      4           38382.255833      7.854417
      5           23761.120000      7.854417
...                        ...           ...
39    95          97174.169167      8.450250
      96          24947.875833      8.450250
      97          23002.670000      8.450250
      98           9089.097500      8.450250
      99            317.189091      7.670091

[929 rows x 2 columns]

In [11]:
#What percent of sales occured at each store type
sales_all = df['weekly_sales'].sum()
sales_A = df[df['type']== 'A']['weekly_sales'].sum()
sales_B = df[df['type']== 'B']['weekly_sales'].sum()
sales_C = df[df['type']== 'C']['weekly_sales'].sum()
#Get proportion for each type
sales_propn_by_type = [sales_A, sales_B, sales_C]/ sales_all
print(sales_propn_by_type)

[0.9097747 0.0902253 0.       ]


In [12]:
#calculations with .groupby()
sales_by_type = df.groupby('type')['weekly_sales'].sum()
sales_by_type_is_holiday = df.groupby(['type','is_holiday'])['weekly_sales'].sum()
print(sales_by_type_is_holiday)

type  is_holiday
A     False         2.336927e+08
      True          2.360181e+04
B     False         2.317678e+07
      True          1.621410e+03
Name: weekly_sales, dtype: float64


In [13]:
#multiple grouped summaries
import numpy as np
sales_stats = df.groupby('weekly_sales').agg([min, max, np.mean, np.median])
print(sales_stats)

             Unnamed: 0                       store                   \
                    min   max    mean  median   min max  mean median   
weekly_sales                                                           
-1098.00           1420  1422  1421.0  1421.0     2   2   2.0    2.0   
-798.00            4110  4115  4112.5  4112.5    10  10  10.0   10.0   
-698.00             517  3217  1867.0  1867.0     1   6   3.5    3.5   
-598.00            7718  7719  7718.5  7718.5    20  20  20.0   20.0   
-498.00            5912  5912  5912.0  5912.0    14  14  14.0   14.0   
...                 ...   ...     ...     ...   ...  ..   ...    ...   
 213042.66         6237  6237  6237.0  6237.0    14  14  14.0   14.0   
 214383.07         6202  6202  6202.0  6202.0    14  14  14.0   14.0   
 218559.54         4281  4281  4281.0  4281.0    10  10  10.0   10.0   
 232558.51         4271  4271  4271.0  4271.0    10  10  10.0   10.0   
 293966.05         6201  6201  6201.0  6201.0    14  14  14.0   

In [14]:
unemp_fuel_stats = df.groupby('type')[['unemployment','fuel_price_usd_per_l']].agg([min, max,np.mean, np.median])
print(unemp_fuel_stats)

     unemployment                         fuel_price_usd_per_l            \
              min    max      mean median                  min       max   
type                                                                       
A           3.879  8.992  7.972611  8.067             0.664129  1.107410   
B           7.170  9.765  9.279323  9.199             0.760023  1.107674   

                          
          mean    median  
type                      
A     0.744619  0.735455  
B     0.805858  0.803348  


In [15]:
#pivot tables in pandas, values = column to summarize and index = groupby; pivot_table always takes the mean value for each 
#-- group by default
df.pivot_table(values='weekly_sales', index = 'type')
df.pivot_table(values='weekly_sales', index = 'type', aggfunc=[np.mean, np.median])
df.pivot_table(values='unemployment', index='store', columns = 'type')
df.pivot_table(values='weekly_sales', index='type', columns='is_holiday')
df.pivot_table(values='unemployment', index='store', columns = 'type', fill_value =0)
df.pivot_table(values='unemployment', index='store', columns = 'type', fill_value=0, margins=True)#summary statistic

type          A         B       All
store                              
1      7.849257  0.000000  7.849257
2      8.156532  0.000000  8.156532
4      7.535630  0.000000  7.535630
6      7.034975  0.000000  7.034975
10     0.000000  9.279323  9.279323
13     7.941028  0.000000  7.941028
14     8.795390  0.000000  8.795390
19     8.118392  0.000000  8.118392
20     7.673268  0.000000  7.673268
27     8.037213  0.000000  8.037213
31     8.157491  0.000000  8.157491
39     8.425687  0.000000  8.425687
All    7.972611  9.279323  8.082009

In [17]:
df.pivot_table(values='unemployment', index='type', columns = 'is_holiday', fill_value=0, margins=True)
df.pivot_table(values='unemployment', index = 'type', aggfunc=[np.mean, np.median])

mean       median
     unemployment unemployment
type                          
A        7.972611        8.067
B        9.279323        9.199